In [22]:
# Block 1: Importing Libraries
import tarfile
import resampy
import pandas as pd
import librosa
import numpy as np
import os
import shutil
import concurrent.futures
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from concurrent.futures import ThreadPoolExecutor
from torch.optim.lr_scheduler import CosineAnnealingLR
import albumentations as A
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

GPU: NVIDIA GeForce RTX 4080 Laptop GPU is available.


In [23]:
# Block 2: Extracting Data
def extract_tar(tar_file, target_dir):
    if os.path.exists(target_dir):
        user_input = input(f"The directory '{target_dir}' already exists. Do you want to skip extraction? (y/n): ")
        if user_input.lower() == 'y':
            print(f"Skipping extraction of {tar_file}.")
            return
        else:
            print(f"Overwriting the existing directory '{target_dir}'.")
            shutil.rmtree(target_dir)
    with tarfile.open(tar_file, 'r') as tar:
        tar.extractall(target_dir)
    # Remove residue "._" hidden files from the inner folder
    inner_folder = os.path.join(target_dir, os.path.splitext(os.path.basename(tar_file))[0])
    for root, dirs, files in os.walk(inner_folder):
        for file in files:
            if file.startswith("._"):
                os.remove(os.path.join(root, file))

extract_tar('train_mp3s.tar', 'train_mp3s')
extract_tar('test_mp3s.tar', 'test_mp3s')
train_labels = np.loadtxt('train_label.txt', dtype=int)

Skipping extraction of train_mp3s.tar.
Skipping extraction of test_mp3s.tar.


In [24]:
# Block 3: Preprocessing Functions

import pickle

def save_preprocessed_data(data, labels, folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    with open(os.path.join(folder_path, 'train_features.pkl'), 'wb') as f:
        pickle.dump(data, f)

    with open(os.path.join(folder_path, 'train_labels.pkl'), 'wb') as f:
        pickle.dump(labels, f)

def load_preprocessed_data(folder_path):
    with open(os.path.join(folder_path, 'train_features.pkl'), 'rb') as f:
        data = pickle.load(f)

    with open(os.path.join(folder_path, 'train_labels.pkl'), 'rb') as f:
        labels = pickle.load(f)

    return data, labels
import concurrent.futures

def extract_mfcc(audio, sample_rate):
    return librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)

def extract_mel_spec(audio, sample_rate):
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_scaled = np.mean(mel_spec_db.T, axis=0)
    return mel_spec_scaled

def extract_tonnetz(audio, sample_rate):
    return librosa.feature.tonnetz(y=audio, sr=sample_rate)

def extract_chroma_stft(audio, sample_rate):
    return librosa.feature.chroma_stft(y=audio, sr=sample_rate)

def extract_chroma_cqt(audio, sample_rate):
    return librosa.feature.chroma_cqt(y=audio, sr=sample_rate)

def extract_chroma_cens(audio, sample_rate):
    return librosa.feature.chroma_cens(y=audio, sr=sample_rate)

def preprocess_audio(file_path_A, file_path_B, augment=False):
    try:
        audio_A, sample_rate_A = librosa.load(file_path_A, res_type='kaiser_fast')
        audio_B, sample_rate_B = librosa.load(file_path_B, res_type='kaiser_fast')
        print(f"Loaded audio files: {file_path_A}, {file_path_B}")

        mel_spec_A = librosa.feature.melspectrogram(y=audio_A, sr=sample_rate_A, n_mels=128)
        mel_spec_db_A = librosa.power_to_db(mel_spec_A, ref=np.max)

        mel_spec_B = librosa.feature.melspectrogram(y=audio_B, sr=sample_rate_B, n_mels=128)
        mel_spec_db_B = librosa.power_to_db(mel_spec_B, ref=np.max)

        # Apply audio augmentation
        if augment:
            mixed_mel_spectrogram = apply_audio_augmentation((mel_spec_db_A, mel_spec_db_B))
        else:
            mixed_mel_spectrogram = mel_spec_db_A

        features_scaled = np.mean(mixed_mel_spectrogram.T, axis=0)
        print(f"Extracted features: {features_scaled.shape}")
        return features_scaled
    except Exception as e:
        print(f"Error processing files: {file_path_A}, {file_path_B}")
        print(f"Error message: {str(e)}")
        return None
def apply_audio_augmentation(mel_spectrograms):
    # Unpack the mel spectrograms
    mel_spectrogram_A, mel_spectrogram_B = mel_spectrograms

    # Get the shape of the mel spectrograms
    F, T, C = mel_spectrogram_A.shape

    # Create a binary mask filled with ones
    M = np.ones((F, T), dtype=int)

    # Frequency masking
    ftimes = random.randint(0, 3)
    for _ in range(ftimes):
        fstart = random.randint(0, F - 1)
        γ = random.uniform(0, 1)
        fend = int(fstart + γ * F)
        M[fstart:fend, :] = 0

    # Time masking
    ttimes = random.randint(0, 3)
    for _ in range(ttimes):
        tstart = random.randint(0, T - 1)
        γ = random.uniform(0, 1)
        tend = int(tstart + γ * T)
        M[:, tstart:tend] = 0

    # Apply the mask to the mel spectrograms
    masked_mel_spectrogram_A = M * mel_spectrogram_A
    masked_mel_spectrogram_B = (1 - M) * mel_spectrogram_B

    # Combine the masked mel spectrograms
    mixed_mel_spectrogram = masked_mel_spectrogram_A + masked_mel_spectrogram_B

    return mixed_mel_spectrogram

def process_file(file_path, augment=False):
    print(f"Processing file: {file_path}")
    features = preprocess_audio(file_path, augment=augment)
    return features

def prepare_data(directory, augment=False):
    file_paths = [os.path.join(directory, f"{i}.mp3") for i in range(len(os.listdir(directory)))]
    file_path_pairs = [(file_paths[i], file_paths[j]) for i in range(len(file_paths)) for j in range(i+1, len(file_paths))]
    
    with ThreadPoolExecutor(max_workers=32) as executor:
        results = list(executor.map(lambda x: process_file(x[0], x[1], augment), file_path_pairs))
    
    features = [feat for feat in results if feat is not None]
    print(f"Processed {len(features)} audio file pairs")
    return np.array(features)

In [25]:
# Block 4: Preparing Data (modified)
folder_path = 'V5.5c2'

try:
    train_features, train_labels = load_preprocessed_data(folder_path)
    print("Loaded preprocessed data from the 'V5' folder.")
except FileNotFoundError:
    train_features_original = prepare_data('train_mp3s/train_mp3s')
    print(f"Original train features shape: {train_features_original.shape}")

    train_features_augmented = prepare_data('train_mp3s/train_mp3s', augment=True)
    print(f"Augmented train features shape: {train_features_augmented.shape}")

    train_features = np.concatenate((train_features_original, train_features_augmented), axis=0)
    print(f"Combined train features shape: {train_features.shape}")

    test_features = prepare_data('test_mp3s/test_mp3s')
    print(f"Test features shape: {test_features.shape}")

    label_encoder = LabelEncoder()
    train_labels_original = label_encoder.fit_transform(train_labels)
    train_labels_augmented = train_labels_original.copy()
    train_labels = np.concatenate((train_labels_original, train_labels_augmented), axis=0)
    print(f"Train labels shape: {train_labels.shape}")

    print(f"Number of training features: {len(train_features)}")
    print(f"Number of training labels: {len(train_labels)}")
    print(f"Number of test features: {len(test_features)}")

    save_preprocessed_data(train_features, train_labels, folder_path)
    print("Saved preprocessed data to the {folder_path} folder.")

if len(train_features) == 0:
    print("No training features available. Please check the data.")

In [ ]:
# Block 5: Model Training and Prediction
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if len(train_features) > 0:
    if len(train_features) != len(train_labels):
        raise ValueError("Number of train features and labels do not match.")

    # Standardize the features
    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_features)
    test_features = scaler.transform(test_features)

    # Create a custom dataset class
    class AudioDataset(Dataset):
        def __init__(self, features, labels):
            self.features = torch.tensor(features, dtype=torch.float32)
            self.labels = torch.tensor(labels, dtype=torch.long)

        def __len__(self):
            return len(self.features)

        def __getitem__(self, index):
            return self.features[index], self.labels[index]

    # Set up cross-validation
    num_folds = 10
    kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

    # Initialize variables to store the best model and its metrics
    best_model = None
    best_accuracy = 0.0
    best_f1 = 0.0
    best_precision = 0.0
    best_recall = 0.0

    # Perform cross-validation
    for fold, (train_indices, val_indices) in enumerate(kfold.split(train_features, train_labels), 1):
        print(f"Fold {fold}/{num_folds}")

        # Create data subsets for the current fold
        train_data = train_features[train_indices]
        train_labels_fold = train_labels[train_indices]
        val_data = train_features[val_indices]
        val_labels_fold = train_labels[val_indices]

        # Create PyTorch datasets
        train_dataset = AudioDataset(train_data, train_labels_fold)
        val_dataset = AudioDataset(val_data, val_labels_fold)

        # Create data loaders
        batch_size = 64
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        # Define the model architecture
        model = nn.Sequential(
            nn.Linear(train_features.shape[1], 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.BatchNorm1d(256),
            nn.Linear(256, 4)
        ).to(device)

        # Define the loss function, optimizer, and scheduler
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
        scheduler = CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6)

        # Train the model
        num_epochs = 155
        for epoch in range(num_epochs):
            model.train()
            for batch_data, batch_labels in train_loader:
                batch_data = batch_data.to(device)
                batch_labels = batch_labels.to(device)

                optimizer.zero_grad()
                outputs = model(batch_data)
                loss = criterion(outputs, batch_labels)
                loss.backward()
                optimizer.step()

            model.eval()
            with torch.no_grad():
                predictions = []
                true_labels = []
                for batch_data, batch_labels in val_loader:
                    batch_data = batch_data.to(device)
                    batch_labels = batch_labels.to(device)

                    outputs = model(batch_data)
                    _, predicted = torch.max(outputs, 1)
                    predictions.extend(predicted.cpu().numpy())
                    true_labels.extend(batch_labels.cpu().numpy())

                accuracy = accuracy_score(true_labels, predictions)
                f1 = f1_score(true_labels, predictions, average='weighted')
                precision = precision_score(true_labels, predictions, average='weighted')
                recall = recall_score(true_labels, predictions, average='weighted')
                print(f"Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

            scheduler.step()

        # Update the best model if the current model has higher F1 score
        if f1 > best_f1:
            best_model = model
            best_accuracy = accuracy
            best_f1 = f1
            best_precision = precision
            best_recall = recall

    # Evaluate the best model on the test set
    test_features = torch.tensor(test_features, dtype=torch.float32).to(device)
    best_model.eval()
    with torch.no_grad():
        outputs = best_model(test_features)
        _, predicted_labels = torch.max(outputs, 1)
        predicted_labels = label_encoder.inverse_transform(predicted_labels.cpu().numpy())

    submission = pd.DataFrame({'id': range(len(predicted_labels)), 'category': predicted_labels})
    submission.to_csv('submission.csv', index=False)

    print(f"Best model metrics:")
    print(f"Accuracy: {best_accuracy:.4f}")
    print(f"F1 Score: {best_f1:.4f}")
    print(f"Precision: {best_precision:.4f}")
    print(f"Recall: {best_recall:.4f}")

else:
    print("No training features available. Please check the data.")

Fold 1/10
Epoch [1/200], Validation Accuracy: 0.7014, F1 Score: 0.7018, Precision: 0.7045, Recall: 0.7014
Epoch [2/200], Validation Accuracy: 0.7195, F1 Score: 0.7192, Precision: 0.7198, Recall: 0.7195
Epoch [3/200], Validation Accuracy: 0.7183, F1 Score: 0.7178, Precision: 0.7180, Recall: 0.7183
Epoch [4/200], Validation Accuracy: 0.7246, F1 Score: 0.7227, Precision: 0.7247, Recall: 0.7246
Epoch [5/200], Validation Accuracy: 0.7279, F1 Score: 0.7282, Precision: 0.7289, Recall: 0.7279
Epoch [6/200], Validation Accuracy: 0.7405, F1 Score: 0.7410, Precision: 0.7428, Recall: 0.7405
Epoch [7/200], Validation Accuracy: 0.7296, F1 Score: 0.7300, Precision: 0.7308, Recall: 0.7296
Epoch [8/200], Validation Accuracy: 0.7279, F1 Score: 0.7276, Precision: 0.7277, Recall: 0.7279
Epoch [9/200], Validation Accuracy: 0.7351, F1 Score: 0.7343, Precision: 0.7351, Recall: 0.7351
Epoch [10/200], Validation Accuracy: 0.7456, F1 Score: 0.7456, Precision: 0.7469, Recall: 0.7456
Epoch [11/200], Validation Ac